In [24]:
# pip uninstall twint

In [25]:
# pip install --user --upgrade git+https://github.com/twintproject/twint.git@origin/master#egg=twint

In [26]:
# pip install --user --upgrade git+https://github.com/himanshudabas/twint.git@origin/twint-fixes#egg=twint

In [3]:
import numpy as np
import pandas as pd
import twint

import time
from datetime import datetime
from datetime import timedelta, date

from os import listdir
from os.path import isfile, join

from collections import Counter
from ast import literal_eval

import re

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_colwidth', 199)
pd.options.display.float_format = '{:.2f}'.format

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

import nest_asyncio
nest_asyncio.apply()

In [28]:
def clean_tweets(df):
    local_df = df
    
    #drop redundant cols
    for redundant_col in ['created_at', 'timezone', 'cashtags', 'user_id_str', 'photos', 'video', 'thumbnail', 'translate', 'trans_src', 'trans_dest', 'name', 'search']:
        try:
            local_df.drop([redundant_col], axis=1, inplace=True)
        except:
            pass
    
    #convert date to datetime
    local_df['date'] = pd.to_datetime(local_df['date'])
    
    return local_df

In [29]:
def find_tweets_in_date_range(search_terms, start_date, end_date, LOCAL_SAVE_PATH_ROOT):
    '''    
    search_terms = list of strings (or singleton) of search terms that will be in the tweet
    
    returns a pandas dataframe of tweets
    '''
    
    search_string = " OR ".join(search_terms)
    
    c = twint.Config()
    
#     date range of search
    c.Since = start_date
    c.Until = end_date
    
    c.Search = search_string
    
    # only collect a certain number of tweets
#     c.Limit = num_tweets

    # Don't print output
    c.Hide_output = True
    
    # find shadow-banned accounts too - this apparently slows things down considerably
    c.Profile_full = True

#     c.Output = f"{num_tweets} tweets - {start_date} to {end_date}.csv"
    c.Pandas = True

    twint.run.Search(c)

#     return twint.storage.panda.Tweets_df
    return clean_tweets(twint.storage.panda.Tweets_df)

#     clean_tweets(twint.storage.panda.Tweets_df).to_csv(LOCAL_SAVE_PATH_ROOT, index=False)

In [30]:
# test = find_tweets_in_date_range(search_terms=test_search_terms, start_date='2020-01-01', end_date='2021-07-07', LOCAL_SAVE_PATH_ROOT=0)

#believescience


In [23]:
non_CT_words_and_tags = [
    'essentialworkers',
    'pfizerproud',
    'vaccineswork',
    'sciencewillwin'
]

non_CT_words_and_tags = non_CT_words_and_tags + [f'#{x}' for x in non_CT_words_and_tags]

test_search_terms = non_CT_words_and_tags

In [32]:
# SAVE_PATH_ROOT = r"C:/Users/mikha/Dropbox/mikhael_misc/Projects/594/Twitter-Conspiracies/Pulling Twitter Data/Pulling non-CT Tweets/non-ct tweets by date/"

# date_ranges = []

# sixdays = timedelta(days=6)

# today_str = date.today().strftime('%m-%d-%Y')

# for date in pd.date_range(start='2020-01-01', end='2021-07-07', freq='1W'):
#     date_tuple = ((date.strftime('%Y-%m-%d'), (date+sixdays).strftime('%Y-%m-%d') ))
    
#     print(date_tuple)
    
#     date_ranges.append(date_tuple)

# for date_tup in date_ranges():
#     find_tweets_in_date_range(search_terms=test_search_terms,
#                               start_date=date_tup[0],
#                               end_date=date_tup[1],
#                               LOCAL_SAVE_PATH_ROOT=SAVE_PATH_ROOT + f'{date_tup[0]} to {date_tup[1]}.csv')

In [33]:
test = find_tweets_in_date_range(search_terms=test_search_terms, start_date='2020-01-01', end_date='2021-07-07', LOCAL_SAVE_PATH_ROOT=0)

test.to_csv(r"#essentialworkers, #pfizerproud, #vaccineswork, #sciencewillwin.csv", index=False)

# Clean Non-CT Tweets

## Remove Users Who Are Also Flagged for CT

In [29]:
def label_overlapped_users(df1, df2):
    '''
    Both DataFrames must have user ID (or something like that) as index
    '''
    
    intersection_of_users = np.intersect1d(df1.index, df2.index)
    
    df1['In CT and Non-CT DFs'] = 0
    df2['In CT and Non-CT DFs'] = 0
    
    df1['In CT and Non-CT DFs'].loc[intersection_of_users] = 1
    df2['In CT and Non-CT DFs'].loc[intersection_of_users] = 1

    return df1, df2
    

In [30]:
# MASTER_USER_PATH = r"C:/Users/crackcocaine69xxx/Python Stuff/594/Twint/Geo Cross Referencing/Master User List - GEOLOCATED.csv"
MASTER_USER_PATH = "C:/Users/mikha/Dropbox/mikhael_misc/Projects/594/Twitter-Conspiracies/Geo Cross Referencing/Master User List - GEOLOCATED.csv"


ALL_CT_USERS = pd.read_csv(MASTER_USER_PATH).set_index('ID')

In [31]:
non_CT_tweets_2 = pd.read_csv(r"#essentialworkers, #pfizerproud, #vaccineswork, #sciencewillwin.csv")

non_CT_tweets_2 = non_CT_tweets_2.set_index('id')

C:\Users\mikha\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3146: DtypeWarning: Columns (6) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [32]:
non_CT_tweets_2, ALL_CT_USERS = label_overlapped_users(non_CT_tweets_2, ALL_CT_USERS)

C:\Users\mikha\Anaconda\lib\site-packages\pandas\core\indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [33]:
non_CT_tweets_2.head(2)

,conversation_id,date,time,user_id,username,place,tweet,language,mentions,urls,replies_count,retweets_count,likes_count,hashtags,link,retweet,quote_url,near,geo,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,In CT and Non-CT DFs
id,,,,,,,,,,,,,,,,,,,,,,,,,,
1313309934143123459,1313309934143123459,2020-10-05,19:47:43,1080845914527289346,SirfSach01,NaN,कोरोना का कहर जारी: दुनिया की 10 फीसदी आबादी जानलेवा वायरस से संक्रमित- @WHO #Coronavirus #WHO #Covid19 #VaccinesWork https://t.co/YfNyGFfphd,hi,"[{'screen_name': 'who', 'name': 'world health organization (who)', 'id': '14499829'}]",['https://www.sirfsach.in/cutting-edge/who-said-ten-percent-of-world-population-may-have-had-coronavirus/32653/'],0,0,0,"['coronavirus', 'who', 'covid19', 'vaccineswork']",https://twitter.com/SirfSach01/status/1313309934143123459,False,NaN,nan,nan,nan,nan,nan,nan,[],nan,0
1313308930131529728,1313258094194036736,2020-10-05,19:43:43,756080400,CrockettDebra,NaN,"@edyong209 @ClintSmithIII @AdamHSays @elainejgodfrey I'm so angry about his irresponsible behavior. Once again, people of color are adversely impacted by Covid. #EssentialWorkers",en,[],[],0,3,2,['essentialworkers'],https://twitter.com/CrockettDebra/status/1313308930131529728,False,NaN,nan,nan,nan,nan,nan,nan,"[{'screen_name': 'edyong209', 'name': 'Ed Yong', 'id': '19767193'}, {'screen_name': 'ClintSmithIII', 'name': 'Clint Smith', 'id': '472587261'}, {'screen_name': 'AdamHSays', 'name': 'adam harris',...",nan,0


## Clean and Label Tweets

### Keyword lists and such

In [34]:
general_conspiracy_hashtags = [
    'plandemic',
    'scamdemic',
    'covidhoax',
    'nwo',
    'covid1984',
    'plandemia',
    'agenda21',
    'thegreatreset',
    'agenda2030',
    'newworldorder',
    'wakeupamerica',
#     'wakeup',
    'openamericanow',
    'firefauci',
    'wwg1wga',
    'qanon',
    'coronahoax',
    
    'essentialworkers',
    'pfizerproud',
    'vaccineswork',
    'sciencewillwin'
]

keywords = [
    'plandemic',
    'scamdemic',
    'covidhoax',
    'covid hoax',
    'covid1984',
    'plandemia',
    'new world order',
    'wake up america',
    'open america now',
    'fire fauci',
    'wwg1wga',
    'qanon',
    'coronahoax',
    'corona hoax',
]

CT_link_list = ['zerohedge.com', 'infowars.com', 'principia-scientific.com',
'tx.voice-truth.com', 'humansarefree.com', 'activistpost.com'
'gnews.org', 'wakingtimes.com', 'brighteon.com','thewallwillfall.org','sott.net',]


hashtag_set = set(['#' + tag.upper() for tag in general_conspiracy_hashtags])
keyword_set = set([x.upper() for x in keywords])

re_escape_keywords = '|'.join([re.escape(word) for word in keywords])
re_escape_links = '|'.join([re.escape(link) for link in CT_link_list])

## Define Functions for Cleaning

In [35]:
def hashtag_in_list(list_of_hashtags_in_tweet):
    return any(hashtag.upper() in [tag.upper() for tag in list_of_hashtags_in_tweet] for hashtag in general_conspiracy_hashtags)

In [36]:
non_CT_words_and_tags = [x.upper() for x in non_CT_words_and_tags]

In [37]:
def clean_tweet(tweet):
    '''
    input = tweet (str)
    output = cleaned_tweet(str)
    '''
    
    return ' '.join([x.replace('#','') for x in tweet.split() if not (x.startswith(('http','@')) or x.upper() in keyword_set or x.upper() in hashtag_set or x.upper() in non_CT_words_and_tags)])

In [38]:
def label_CT_tweets(df):
    
    return ( df['tweet'].str.contains(re_escape_keywords, case=False) | df['urls'].str.contains(re_escape_links, case=False) | df['hashtags'].apply(hashtag_in_list)).astype(int)


In [39]:
non_CT_tweets_2['CT Tweet'] = label_CT_tweets(non_CT_tweets_2)

In [40]:
non_CT_tweets_2 = non_CT_tweets_2[non_CT_tweets_2['language']=='en']

In [41]:
non_CT_tweets_2.to_csv(r"Semi-Cleaned #essentialworkers, #pfizerproud, #vaccineswork, #sciencewillwin.csv")

In [42]:
non_CT_tweets_2 = pd.read_csv(r"Semi-Cleaned #essentialworkers, #pfizerproud, #vaccineswork, #sciencewillwin.csv")

# Final Cleaning

In [43]:
non_CT_tweets_2['Cleaned Tweet'] = non_CT_tweets_2['tweet'].apply(clean_tweet)

In [45]:
# non_CT_tweets_2[(non_CT_tweets_2['Cleaned Tweet'].map(len) > 2)][['Cleaned Tweet', 'CT Tweet']].to_csv(r'#essentialworkers, #pfizerproud, #vaccineswork, #sciencewillwin - CLEANED FOR BERT.csv')

non_CT_tweets_2[(non_CT_tweets_2['Cleaned Tweet'].map(len) > 2) & (non_CT_tweets_2['CT Tweet']==0)][['Cleaned Tweet', 'CT Tweet']].rename(columns={'Cleaned Tweet':'sentence1', 'CT Tweet':'label'}).to_csv(r'#essentialworkers, #pfizerproud, #vaccineswork, #sciencewillwin - CLEANED FOR BERT.csv')

In [75]:
# non_CT_tweets_2[(non_CT_tweets_2['Cleaned Tweet'].map(len) > 2) & (non_CT_tweets_2['CT Tweet']==0)][['Cleaned Tweet', 'CT Tweet']].rename(columns={'Cleaned Tweet':'sentence1', 'CT Tweet':'label'})

In [51]:
len(non_CT_tweets_2[(non_CT_tweets_2['Cleaned Tweet'].map(len) > 2)])

477522

In [49]:
(non_CT_tweets_2['Cleaned Tweet'].str.split().map(len) > 4).sum() / len(non_CT_tweets_2)

0.9846687691875976